In [2]:
from source.chroma_manager import ChromaDBManager, CustomEmbeddingFunction
from tqdm import tqdm
import yaml
import sys
import json

In [3]:
CONFIG_PATH = "config/embedding/all-MiniLM-L6-v2.yaml"
# CONFIG_PATH = "config/embedding/all-mpnet-base-v2.yaml"
# CONFIG_PATH = "config/embedding/multi-qa-mpnet-base-dot-v1.yaml"
# CONFIG_PATH = "config/embedding/paraphrase-multilingual-MiniLM-L12-v2.yaml"

config_path_question = "config/embedding/questions_gen.yaml"

In [4]:
with open(CONFIG_PATH, "r") as file:
    config = yaml.safe_load(file)

CHROMA_PATH = config["chroma_path"]
COLLECTION_NAME = config["collection_name"]
MODEL_NAME = config["model_name"]

with open(config_path_question, "r") as file:
    config_question_db = yaml.safe_load(file)

CHROMA_PATH_QUESTION = config_question_db["chroma_path"]
COLLECTION_NAME_QUESTION = config_question_db["collection_name"]
MODEL_NAME_QUESTION = config_question_db["model_name"]


# Создаём менеджер с кастомной функцией эмбеддингов
embedding_function = CustomEmbeddingFunction(model_name=MODEL_NAME)

# Инициализируем менеджер с кастомными эмбеддингами
model_chroma = ChromaDBManager(
    storage_path=CHROMA_PATH,
    collection_name=COLLECTION_NAME,
    embedding_function=embedding_function
)

# Инициализируем менеджер с вопросами
questions_chroma = ChromaDBManager(
    storage_path=CHROMA_PATH_QUESTION,
    collection_name=COLLECTION_NAME_QUESTION,
)

In [5]:
# Извлечение всех документов
all_documents = questions_chroma.collection.get()
doc_ids = all_documents.get("ids", [])
doc_metadata = all_documents.get("metadatas", [])

In [6]:
len(doc_ids)

549

In [7]:
doc_ids_batch = doc_ids[295:305]
doc_metadata_batch = doc_metadata[295:305]

In [13]:
result = {}

# Прогон идет по БД с вопросами
for doc_id, metadata in tqdm(zip(doc_ids_batch, doc_metadata_batch)): 

    question_1 = eval(metadata['questions'])['question_1']
    question_2 = eval(metadata['questions'])['question_2']
    answer_1 = eval(metadata['questions'])['answer_1']
    answer_2 = eval(metadata['questions'])['answer_2']
    query_results_1 = model_chroma.query(question_1, n_results=3)
    query_results_2 = model_chroma.query(question_2, n_results=3)
    
    position_1, position_2 = False, False

    validation_1 = doc_id in query_results_1['ids'][0]
    if validation_1:
        position_1 = query_results_1['ids'][0].index(doc_id)

    validation_2 = doc_id in query_results_2['ids'][0]
    if validation_2:
        position_2 = query_results_2['ids'][0].index(doc_id)
    
    temp_res = {
        doc_id: {
            "question_1": {
                    "search_ids": query_results_1['ids'],
                    "validation": validation_1,
                    "position": position_1,
                    "question": question_1,
                    "answer": answer_1,
                            },
            "question_2": {
                    "search_ids": query_results_2['ids'],
                    "validation": validation_2,
                    "position": position_2,
                    "question": question_2,
                    "answer": answer_2,
            }
        }
    }
    result.update(temp_res)

output_filemane = "benchmark/output_RAG/questions.json"

with open(output_filemane, "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

10it [00:00, 28.49it/s]
